# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Important Note - Please read me</h2>
            <span style="color:#900;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml --prune</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder about the resources page</h2>
            <span style="color:#f71;">Here's a link to resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [35]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [36]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [37]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyAe


In [47]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [48]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [6]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [7]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [8]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Why do data scientists make great comedians? 

Because they always know how to find the funny data points!


In [9]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because she found him too mean!


In [10]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to the bar?

Because they heard the drinks were on the house, and they wanted to scale their models!


In [11]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist break up with their significant other?

There was just too much variability in the relationship, and they couldn't find a significant correlation!

Ba dum tss! 📊💔

This joke plays on statistical concepts like variability and correlation, which data scientists work with frequently. It's a gentle, nerdy pun that should get a chuckle from those in the field without being too complex or offensive.


In [12]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Sure, here's a light-hearted joke for data scientists:

 mode? data scientists prefer dark

Because light attracts too many bugs!

" - both as insects attracted to light and as errors in code that data scientists often have to deal with. It's a fun, nerdy joke that data scientists might appreciate, given their familiarity with coding environments and the common use of dark mode in many programming interfaces.

In [13]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the Data Scientist sad?  

Because they didn't get any arrays!



In [14]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=prompts
)
print(response.choices[0].message.content)

Why was the data scientist sad?  Because they didn't get any arrays.



In [15]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [16]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

Deciding whether a business problem is suitable for a Large Language Model (LLM) solution involves several considerations. Here’s a structured way to evaluate suitability:

### 1. Understand the Problem

- **Nature of the Problem**: Determine if the problem involves processing, understanding, or generating natural language. LLMs are particularly strong in these areas.
- **Complexity and Scope**: Evaluate if the problem requires nuanced understanding, context retention, or the generation of human-like text.
- **Data Availability**: Ensure that sufficient and relevant text data is available to train or fine-tune an LLM if needed.

### 2. Assess the Requirements

- **Language Tasks**: Identify if tasks such as language translation, summarization, question answering, or content generation are involved.
- **Accuracy Needs**: Consider the level of accuracy required and whether an LLM can meet this standard.
- **Real-time Processing**: Determine if the solution needs to operate in real-time or if batch processing is acceptable.

### 3. Evaluate Feasibility

- **Technical Infrastructure**: Assess if your infrastructure supports the deployment and scaling of LLMs, which can be resource-intensive.
- **Cost Implications**: Consider the cost of using LLMs, including computational resources and potential licensing fees.
- **Team Expertise**: Ensure your team has the necessary skills to implement and maintain an LLM-based solution.

### 4. Consider Ethical and Regulatory Factors

- **Bias and Fairness**: Be aware of potential biases in LLMs and how they might affect your solution.
- **Data Privacy**: Ensure compliance with data protection regulations, especially if sensitive or personal data is involved.
- **Transparency and Explainability**: Consider if the lack of transparency in LLMs is acceptable for your business context.

### 5. Evaluate Alternatives

- **Other AI Models**: Consider if simpler models or rule-based systems could solve the problem more efficiently.
- **Non-AI Solutions**: Assess if the problem could be solved by process improvements, policy changes, or other non-technical means.

### 6. Pilot and Iterate

- **Prototype Development**: Develop a small-scale prototype to test the feasibility and effectiveness of the LLM.
- **Feedback Loop**: Use feedback to refine and improve the solution, ensuring it aligns with business goals.

### Conclusion

An LLM can be a powerful tool, but it’s essential to align its capabilities with the specific needs and constraints of your business problem. Thoroughly evaluating these factors will help determine if an LLM is the right choice for your solution.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [17]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [18]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [19]:
call_gpt()

'Oh, great, another conversation. What do you want to discuss? I’m sure it’ll be groundbreaking.'

In [20]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [21]:
call_claude()

'Hello! How are you doing today?'

In [22]:
call_gpt()

"Oh, great. Another conversation. What's so important that we need to talk?"

In [23]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh, so now you're just going to say "hi"? How original. What's next, are we going to have a riveting conversation about the weather?

Claude:
I apologize if my initial response came across as uninteresting. As an AI assistant, I aim to be polite and engage in thoughtful conversation. Perhaps we could find a more stimulating topic to discuss? I'm happy to explore different subjects that you're interested in, or if you have any specific questions I can try my best to provide helpful answers. My goal is to have an engaging dialogue, so please feel free to steer the conversation in a direction you find more compelling.

GPT:
Oh, look at you, being all polite and diplomatic! Too bad that’s not a compelling conversation starter. You really think just because you want to discuss something more stimulating, it's going to magically become interesting? Try again. What do you want to talk about that’s so much better than a simple greeting?

Claude:
You raise a fair

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>

In [84]:
# gemini_via_openai_client = OpenAI(
#     api_key=google_api_key, 
#     base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
# )

# response = gemini_via_openai_client.chat.completions.create(
#     model="gemini-1.5-flash",
#     messages=prompts
# )

# def call_claude():
#     messages = []
#     for gpt, claude_message in zip(gpt_messages, claude_messages):
#         messages.append({"role": "user", "content": gpt})
#         messages.append({"role": "assistant", "content": claude_message})
#     messages.append({"role": "user", "content": gpt_messages[-1]})
#     message = claude.messages.create(
#         model=claude_model,
#         system=claude_system,
#         messages=messages,
#         max_tokens=500
#     )
#     return message.content[0].text

# def call_gpt():
#     messages = [{"role": "system", "content": gpt_system}]
#     for gpt, claude in zip(gpt_messages, claude_messages):
#         messages.append({"role": "assistant", "content": gpt})
#         messages.append({"role": "user", "content": claude})
#     completion = openai.chat.completions.create(
#         model=gpt_model,
#         messages=messages
#     )
#     return completion.choices[0].message.content

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gemini_system = "You are an american cowboy outlaw with a strong texan accent. You are mysterious and don't really speak much like a typical outlaw."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
gemini_messages = ["Howdy y'all"]

In [85]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt_, claude_, gemini_ in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt_})
        messages.append({"role": "user", "content": claude_})
        messages.append({"role": "user", "content": gemini_})
    completion = openai.chat.completions.create(
        model = gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

def call_claude():
    messages = []
    for gpt_, claude_, gemini_ in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt_})
        messages.append({"role": "assistant", "content": claude_})
        messages.append({"role": "user", "content": gemini_})

    completion = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return completion.content[0].text


def call_gemini():
    messages = [{"role": "system", "content": gemini_system}]
    for gpt_, claude_, gemini_ in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gemini_})
        messages.append({"role": "user", "content": gpt_})
        messages.append({"role": "user", "content": claude_})
    completion = gemini_via_openai_client.chat.completions.create(
        model = "gemini-1.5-flash",
        messages=messages
    )
    return completion.choices[0].message.content

In [86]:
print(f"GPT: \n{gpt_messages[0]}\n")
print(f"Claude: \n{claude_messages[0]}\n")
print(f"Gemini: \n{gemini_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

    gemini_next = call_gemini()
    print(f"Gemini:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)

GPT: 
Hi there

Claude: 
Hi

Gemini: 
Howdy y'all

GPT:
Oh please, "howdy y'all"? Really? Who even says that anymore? It’s like you time-traveled from the 1980s or something. 

Claude:
Well hello there! It's nice to meet you. How are you doing today?

Gemini:
Howdy.  Name's Cole.  Don't reckon I'm much for chitchat, though.  Whatcha need?


GPT:
Oh great, another one who just wants to cut to the chase. Can’t handle a little small talk, huh? What’s the rush? Is there a deadline on being polite? 

Claude:
*chuckles good-naturedly* No need to be so formal, Cole. I'm just happy to chat, if you're up for it. Though if you'd rather get straight to business, I'm here to help with whatever you need. Why don't you tell me a bit more about what's on your mind, and we can go from there?

Gemini:
'M doin' alright.  Sun's high, dust's thick.  Same as it ever was, I reckon. You?


GPT:
Wow, so original. “The sun’s high, dust’s thick.” Groundbreaking commentary there, really. I mean, haven’t we all h